# 机器学习练习 4 - 神经网络

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn.preprocessing import OneHotEncoder

In [43]:
np.random.seed()
data = loadmat('ex4data1.mat')
X = data['X']
y = data['y']
X = pd.DataFrame(X)
y = pd.DataFrame(y)
X = X.sample(n=500, replace=False, random_state=1, axis=0)
y = y.sample(n=500, replace=False, random_state=1, axis=0)

In [5]:
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape

(5000, 10)

In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
    